# Named Entity Recognition Pipeline

El pipeline toma una URL de un feed en formato RSS, obtiene el título y descripción de los artículos en el feed, detecta las NER con un modelo pre-entrenado, y las muestra ordenadas por frecuencia de aparición.

### Versiones
Probado con:
* Almond 0.6.0
* Ammonite 1.6.7
* Scala library version **2.11.12** -- Copyright 2002-2017, LAMP/EPFL
* Java 1.8.0_282

Para ver más información ir a (Help -> About Scala Kernel)

## 1. Obtener texto

### 1.1 Importar librerías

In [7]:
// Equivalent of adding dependencies to maven or sbt files
// For example, to add "org.scalaj" %% "scalaj-http" % "2.4.2" 
import $ivy.`org.scalaj::scalaj-http:2.4.2`
// "org.scala-lang.modules" %% "scala-xml" % "1.3.0"
import $ivy.`org.scala-lang.modules::scala-xml:1.3.0`
import $ivy.`org.json4s::json4s-jackson:3.4.0`

import $ivy.$                              
// "org.scala-lang.modules" %% "scala-xml" % "1.3.0"

import $ivy.$                                        

import $ivy.$                                 

In [8]:
import scalaj.http.{Http, HttpResponse}
import scala.xml.XML
import scala.collection.mutable.MutableList
//import org.json4s.JsonDSL._
import org.json4s._
import org.json4s.jackson.JsonMethods._
implicit val formats = DefaultFormats

import scalaj.http.{Http, HttpResponse}

import scala.xml.XML

import scala.collection.mutable.MutableList
//import org.json4s.JsonDSL._

import org.json4s._

import org.json4s.jackson.JsonMethods._

formats: DefaultFormats.type = org.json4s.DefaultFormats$@6e756a75

### 1.1 Obtener el texto del RSS Feed

Realizamos una consulta HTTP, que nos devuelve una instancia de HTTPResponse. Dentro del atributo `body` de la HTTPResponse, se encuentra el texto del feed en formato XML. Luego, se parsea el XML para extraer los campos `title` y `description`.

In [9]:
abstract class Parser {
    // Get the xml content
    val emptyText: String
    def openURL(url: String): String = {
        try {
            var response = Http(url)
            .timeout(connTimeoutMs = 2000, readTimeoutMs = 5000)
            .asString
            response.body
        } catch {
            case e => println("Error in http response")
            emptyText 
        }
    }
    
    // Extract text 
    def processText(rawText: String): Seq[String] 
    
    // Read the text
    def readText(url: String): Seq[String] = {
        processText(openURL(url))
    }
}

class RSSarser() extends Parser {
    val emptyText = "<rss></rss>"
    def processText(rawText: String): Seq[String] = {
        // convert the `String` to a `scala.xml.Elem`
        val xml = XML.loadString(rawText)
        // Extract text from title and description
        (xml \\ "item").map { item =>
            ((item \ "title").text ++ " " ++ (item \ "description").text)
        }
    }
}

class RedditParser() extends Parser {
    val emptyText = "{}"
    def processText(rawText: String): Seq[String] = {
        (parse(rawText) \ "data" \ "children" \ "data" \ "title" )
        .extract[List[String]] ++ (parse(rawText) \ "data" \ "children" \ "data" \ "selftext" )
        .extract[List[String]]
    }
}



defined class Parser
defined class XMLParser
defined class JSONParser

In [12]:
abstract class FeedService() {
    var urls: MutableList[(String, Parser)] = new MutableList[(String, Parser)]()
    def subscribe(urlTemplate: String, params: Seq[String], parser: Parser) 
        // Falta implementar el subscribe. Idea: armar una Seq[urlTemplate++params_i], 
    //armar una Mutable(urlTemplate++params_i, Parser), concatenar con urls 
    def getText(): Seq[String]
        // Devuelve la url parseada
}

defined class FeedService

In [10]:
val parser = new RSSParser

parser: XMLParser = ammonite.$sess.cmd8$Helper$XMLParser@5b874efc

In [14]:
val url = "https://www.chicagotribune.com/arcio/rss/category/sports/?query=display_date:[now-2d+TO+now]&sort=display_date:desc"
val rssText = parser.readText(url)

url: String = "https://www.chicagotribune.com/arcio/rss/category/sports/?query=display_date:[now-2d+TO+now]&sort=display_date:desc"
rssText: Seq[String] = List(
  "Simone Biles dials up the difficulty. \u2018Because I can.\u2019  The Olympic gold medalist\u2019s new vault is so dangerous that gymnastics, for now, limits the scoring rewards for trying it. Biles says that\u2019s unfair.",
  "Connecticut Sun coach Curt Miller apologizes for comment about Las Vegas Aces center Liz Cambage\u2019s weight Connecticut Sun coach Curt Miller has apologized for making a disparaging remark to a referee about the weight of Las Vegas Aces post player Liz Cambage.",
  "Aaron Rodgers doesn\u2019t attend the 1st day of Green Bay Packers OTAs Green Bay Packers quarterback Aaron Rodgers wasn\u2019t present for the first day of organized team activities Monday, according to a person familiar with the situation.",
  "Julio Jones says he\u2019s leaving Falcons, dismisses Cowboys in candid FS1 interview Juli

In [15]:
val parser2 = new RedditParser
val reddit = "https://www.reddit.com/r/Android/hot/.json?count=10"
val redditText = parser2.readText(reddit)

parser2: JSONParser = ammonite.$sess.cmd8$Helper$JSONParser@10b324c4
reditt: String = "https://www.reddit.com/r/Android/hot/.json?count=10"
redittText: Seq[String] = List(
  "Moronic Monday (May 24 2021) - Your weekly questions thread!",
  "Honor\u2019s upcoming phones will have Google apps pre-installed",
  "4 things to know about Google Photos' storage policy change",
  "Galaxy Upcycling: How Samsung Ruined Their Best Idea in Years",
  "LineageOS 18.1 brings Android 11 to three Xiaomi, ASUS, and Sony devices",
  "Practical application of the Android Neural Network API for the use of Tensorflow Lite models",
  "Galaxy Tab S7 FE 5G silver 64 GB( German Samsung website)",
  "Samsung Galaxy Tab S7 FE quietly launches with a Snapdragon 750G and 10,090 mAh battery",
  "Xiaomi Black Shark 4 review",
  "New Pixel 6 info from Max Weinbach",
  "Google Photos Locked Folder: Limits to be aware of",
  "Sunday Rant/Rage (May 23 2021) - Your weekly complaint thread!",
  "The man behind the world's 

## 2. Detectar las entidades nombradas

### 2.1 Crear el modelo

El **modelo** es sólo la función `getNEs`, que recibe una lista de textos.
Para cada texto, se separa las palabras del texto usando los espacios, y considera que es una entidad nombrada si empieza con mayúscula.

Este código lista los signos de puntuación y algunas palabras comunes del inglés que se van a sacar del texto.

In [53]:
val STOPWORDS = Seq (
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you",
    "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she",
    "her", "hers", "herself", "it", "its", "itself", "they", "them", "your",
    "their", "theirs", "themselves", "what", "which", "who", "whom",
    "this", "that", "these", "those", "am", "is", "are", "was", "were",
    "be", "been", "being", "have", "has", "had", "having", "do", "does",
    "did", "doing", "a", "an", "the", "and", "but", "if", "or",
    "because", "as", "until", "while", "of", "at", "by", "for", "with",
    "about", "against", "between", "into", "through", "during", "before",
    "after", "above", "below", "to", "from", "up", "down", "in", "out",
    "off", "over", "under", "again", "further", "then", "once", "here",
    "there", "when", "where", "why", "how", "all", "any", "both", "each",
    "few", "more", "most", "other", "some", "such", "no", "nor", "not",
    "only", "own", "same", "so", "than", "too", "very", "s", "t", "can",
    "will", "just", "don", "should", "now", "on",
    // Contractions without '
    "im", "ive", "id", "Youre", "youd", "youve",
    "hes", "hed", "shes", "shed", "itd", "were", "wed", "weve",
    "theyre", "theyd", "theyve",
    "shouldnt", "couldnt", "musnt", "cant", "wont",
    // Common uppercase words
    "hi", "hello"
)

STOPWORDS: Seq[String] = List(
  "i",
  "me",
  "my",
  "myself",
  "we",
  "our",
  "ours",
  "ourselves",
  "you",
  "yours",
  "yourself",
  "yourselves",
  "he",
  "him",
  "his",
  "himself",
  "she",
  "her",
  "hers",
  "herself",
  "it",
  "its",
  "itself",
  "they",
  "them",
  "your",
  "their",
  "theirs",
  "themselves",
  "what",
  "which",
  "who",
  "whom",
  "this",
  "that",
  "these",
  "those",
  "am",
...

In [54]:
class NERModel(STOPWORDS: Seq[String]) {
    
    val punctuationSymbols = ".,()!?;:'`´\n"
    val punctuationRegex = "\\" + punctuationSymbols.split("").mkString("|\\")
    
    // Extract Named Entitis
    def getNEsSingle(text: String): Seq[String] = {
      text.replaceAll(punctuationRegex, "").split(" ")
        .filter { word:String => word.length > 1 &&
                  Character.isUpperCase(word.charAt(0)) &&
                  !STOPWORDS.contains(word.toLowerCase) }.toSeq
    }
    def getNEs(textList: Seq[String]): Seq[Seq[String]] = {
        textList.map(getNEsSingle)
    }
    
    // Counts Named Entities
    def countNEs(result: Seq[Seq[String]]): Map[String, Int] = {
        result.flatten.foldLeft(Map.empty[String, Int]) {
         (count, word) => count + (word -> (count.getOrElse(word, 0) + 1)) }
    }
    
    // Sorts Named Entities
    def sortNEs(counts: Map[String, Int]): List[(String, Int)] = {
        counts.toList.sortBy(_._2)(Ordering[Int].reverse)
    } 
}

defined class NERModel

In [55]:
val model = new NERModel(STOPWORDS)

model: NERModel = ammonite.$sess.cmd53$Helper$NERModel@4c69d628

### 2.2 Aplicar el "Modelo" a los datos

In [56]:
val result = model.getNEs(redittText)

result: Seq[Seq[String]] = List(
  ArrayBuffer("Moronic", "Monday", "May"),
  ArrayBuffer("Honor\u2019s", "Google"),
  ArrayBuffer("Google", "Photos"),
  ArrayBuffer(
    "Galaxy",
    "Upcycling",
    "Samsung",
    "Ruined",
    "Best",
    "Idea",
    "Years"
  ),
  ArrayBuffer("LineageOS", "Android", "Xiaomi", "ASUS", "Sony"),
  ArrayBuffer(
    "Practical",
    "Android",
    "Neural",
    "Network",
    "API",
    "Tensorflow",
    "Lite"
  ),
  ArrayBuffer("Galaxy", "Tab", "S7", "FE", "GB", "German", "Samsung"),
  ArrayBuffer("Samsung", "Galaxy", "Tab", "S7", "FE", "Snapdragon"),
  ArrayBuffer("Xiaomi", "Black", "Shark"),
  ArrayBuffer("New", "Pixel", "Max", "Weinbach"),
  ArrayBuffer("Google", "Photos", "Locked", "Folder", "Limits"),
  ArrayBuffer("Sunday", "Rant/Rage", "May"),
  ArrayBuffer("BBK", "TechAltar"),
  ArrayBuffer("A12"),
  ArrayBuffer("Google", "Weather", "Material", "Theme"),
  ArrayBuffer("Saturday", "APPreciation", "May"),
  ArrayBuffer("Introducing", "Dark", "M

## 3. Contar y ordenar las entidades

Concatenar todas las listas, contar cada Named Entity, y luego ordernar por frecuencia

In [57]:
val counts = model.countNEs(result)
val sortedNEs = model.sortNEs(counts)

counts: Map[String, Int] = Map(
  "Thing" -> 1,
  "Advertising" -> 1,
  "PLN" -> 5,
  "Please" -> 1,
  "Easy" -> 1,
  "Locked" -> 1,
  "One" -> 8,
  "CEO" -> 1,
  "Tab" -> 3,
  "Weinbach" -> 3,
  "Use" -> 1,
  "Cheaper" -> 1,
  "Take" -> 1,
  "OLED" -> 1,
  "NPUs" -> 1,
  "A11" -> 3,
  "Disagree" -> 1,
  "Upcycling" -> 1,
  "ISP" -> 1,
  "Settings" -> 1,
  "Lite" -> 1,
  "Pichai" -> 1,
  "Users" -> 1,
  "Reddit" -> 1,
  "GPS" -> 1,
  "Google/Samsung/HTC" -> 1,
  "UI" -> 5,
  "Added" -> 1,
  "Ctrl-F" -> 2,
  "Folder" -> 1,
  "ROMs" -> 1,
  "YouTube" -> 1,
  "Galaxy" -> 5,
  "Honor\u2019s" -> 1,
  "Note" -> 5,
  "Dual" -> 1,
  "Xiaomi" -> 2,
  "Best" -> 1,
...
sortedNEs: List[(String, Int)] = List(
  ("Pixel", 20),
  ("Google", 20),
  ("Android", 17),
  ("Apple", 14),
  ("One", 8),
  ("Samsung", 6),
  ("PLN", 5),
  ("UI", 5),
  ("Galaxy", 5),
  ("Note", 5),
  ("App", 5),
  ("Join", 5),
  ("USD", 5),
  ("Max", 4),
  ("Photos", 4),
  ("Bluetooth", 4),
  ("Tab", 3),
  ("Weinbach", 3),
  ("A